### Steps
- Use a list of **app_id** to get info from Steam crawler and insert **app_id**, **game_name**, **header_img_url**, **total_positive**, **total_negative**, **total_reviews** into table **games** and insert **game_id**, **review**, **recommended**, **time** into table **reviews**.
- Preprocess **review** in table **reviews** (add_missing_punct, replace_bullets, remove_url, remove_html_tags, normalize_single_quote, remove_non_ascii, remove_ansi_escape_sequences, remove_multi_whitespaces), then tokenize_sent and remove_leading_symbols to insert **review_id**, **sent** into table **sents**. 
- Preprocess **sent** in table **sents** (lowercase, expand contractions, remove_digits, remove_symbols, remove_multi_whitespaces, lemmatize_text, remove_stopwords) to create **sent_prep** in table **sents**.
- Use **sent_prep**, **review_id** in table **sents** to insert **review_prep** in table **reviews** by joining **sent_prep**.
- Use **review_prep** in table **reviews** to calculate special bigrams frequency, get 50 most frequent keywords.
- Insert **kw**, **freq** into table **kws**.
- Embed 50 keywords using S-BERT and cluster them using agglomerative clustering with a distance_threshold=0.6. 
	- Insert **cluster_name** (name of the most frequent keyword in cluster) into table **clusters**.
	- Insert **cluster_id** in table **kws**.
- Loop through **sent_prep** in table **sents**, fuzzy-match each **kw** in table **kws**. 
    - Insert **cluster_id**, **sent_id** in table **clusters_sents**t to link table **clusters** and **sents**.

### Step 1:

- Use a list of **app_id** to get info from Steam crawler and insert **app_id**, **game_name**, **header_img_url**, **total_positive**, **total_negative**, **total_reviews** into table **games** and insert **game_id** (fk), **review**, **recommended**, **time** into table **reviews**.

In [1]:
import requests 
import pandas as pd
import numpy as np

import download_steam_reviews

import sqlite3

import re
from bs4 import BeautifulSoup

import spacy
nlp = spacy.load("en_core_web_sm")

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

from sklearn.cluster import AgglomerativeClustering

from fuzzysearch import find_near_matches

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import flair

sentiment_model = flair.models.TextClassifier.load('sentiment')
sentiment_model_fast = flair.models.TextClassifier.load('sentiment-fast')
senti_analyzer = SentimentIntensityAnalyzer()

import pickle

D:\Anaconda3\envs\steam_insider\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


2021-06-27 18:32:33,006 loading file C:\Users\HuyTran\.flair\models\sentiment-en-mix-distillbert_4.pt


2021-06-28 00:33:02,162 loading file C:\Users\HuyTran\.flair\models\sentiment-en-mix-ft-rnn.pt


In [2]:
# from contractions import contractions
contractions = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

In [3]:
conn = sqlite3.connect('./data/steam_reviews.db') 
cursor = conn.cursor()

In [ ]:
def print_cursor(cursor): 
    print(cursor.execute("""
        select * from games;
    """).fetchall())

In [ ]:
print_cursor(cursor)

In [ ]:
print(cursor.execute("""
        select game_id from games 
        where app_id=367520;
    """).fetchall()[0][0])

In [4]:
# 428550 - Momodora: Reverie under the Moonlight
# 367520 - Hollow Knight
app_ids = [428550, 367520]

In [5]:
### get game_name ###

def get_app_list(): 
    app_list_url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
    resp_data = requests.get(app_list_url)
    return resp_data.json()

def get_name(app_id, app_list): 
    for app in app_list['applist']['apps']: 
        if app['appid'] == app_id: 
            return app['name']

In [6]:
### get header_img_url  

def get_header_img_url(app_id): 
    return f'https://cdn.cloudflare.steamstatic.com/steam/apps/{app_id}/header.jpg'

In [ ]:
### get total_positive, total_negative, total_reviews in Crawler for table games 
### get game_id, review, recommended, time in Crawler for table games 

app_list = get_app_list()

request_params = {
    'language': 'english'
}

# load or download new (maximum ~5000 newest reviews)
load_mode = True

for app_id in app_ids: 
    game_tuples = [] 

    game_name = get_name(app_id, app_list)
    
    header_img_url = get_header_img_url(app_id)
    
    total_positive, total_negative, total_reviews = 0, 0, 0
    
    if load_mode: 
        review_dict = download_steam_reviews.load_review_dict(app_id)['reviews'].values()
    else: 
        review_dict = download_steam_reviews.download_reviews_for_app_id(app_id, 
                                                                     chosen_request_params=request_params, 
                                                                     reviews_limit=5000)[0]['reviews'].values()
    
    review_tuples = []
    
    with conn:
        cursor.execute("""INSERT INTO games (app_id, game_name, header_img_url) VALUES (?, ?, ?);""", 
                       (app_id, game_name, header_img_url))
    
    # get game_id (fk) for table reviews
    game_id = cursor.execute("""SELECT game_id FROM games WHERE app_id=?;""", (app_id,)).fetchone()[0] 
    
    for review_dict_value in review_dict: 
        total_reviews += 1
    
        voted_up = 1 if review_dict_value['voted_up'] else 0
    
        if voted_up: 
            total_positive += 1
        else: 
            total_negative += 1
     
        review = review_dict_value['review']
        recommended = voted_up
        time = review_dict_value['timestamp_updated']
        
        review_tuples.append((review, recommended, time, game_id))
    
    with conn:
        cursor.execute("""UPDATE games SET (total_positive, total_negative, total_reviews) = (?, ?, ?)
                            WHERE game_id=?;""",
                       (total_positive, total_negative, total_reviews, game_id))
    
    with conn:
        cursor.executemany("""INSERT INTO reviews (review, recommended, time, game_id) VALUES 
                                (?, ?, ?, ?);""", review_tuples)    

### Step 2:

- Preprocess **review** in table **reviews** (add_missing_punct, replace_bullets, remove_url, remove_html_tags, normalize_single_quote, remove_non_ascii, remove_ansi_escape_sequences, remove_multi_whitespaces), then tokenize_sent and remove_leading_symbols to insert **review_id**, **sent** into table **sents**. 
- Preprocess **sent** in table **sents** (lowercase, expand contractions, remove_digits, remove_symbols, remove_multi_whitespaces, lemmatize_text, remove_stopwords) to insert **sent_prep** in table **sents**.
- Use **sent_prep**, **review_id** in table **sents** to insert **review_prep** in table **reviews** by joining **sent_prep**.
        

    

In [ ]:
df_reviews = pd.read_sql_query("""SELECT review_id, review, game_id 
                    FROM reviews JOIN games USING(game_id);""", conn)

In [ ]:
df_reviews

In [ ]:
def add_missing_punct(text): 
    return re.sub('([A-Za-z0-9])\s*$', '\g<1>. ', text)


def replace_bullets(text): 
    text = re.sub('([A-Za-z0-9])\s*\n+\s*[+-]?\s*', '\g<1>. ', text)
    text = re.sub('\s*([:+-]+)\s*\n+\s*[+-]?\s*', '. ', text) 
    return text
    
    
# remove url from text
def remove_url(text):
    return re.sub(r"http\S+", ' ', text)


# remove HTML tags
def remove_html_tags(text):
    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    # remove square brackets and characters inside
    text = re.sub('\[(.*?)\]', ' ', text)
    return text


# replace ’ with ' 
def normalize_single_quote(text):
    return re.sub('[’‘]', '\'', text)


# remove non english characters effectively
def remove_non_ascii(text): 
    return text.encode("ascii", errors="ignore").decode()
    
    
# remove ANSI escape sequences
def remove_ansi_escape_sequences(text):
    ansi_escape = re.compile(r'(?:\x1B[@-_]|[\x80-\x9F])[0-?]*[ -/]*[@-~]')
    return ansi_escape.sub('', text)
    
    
# remove multiple whitespaces with single whitespace
def remove_multi_whitespaces(text): 
    return re.sub('\s+', ' ', text.strip())

In [306]:
def remove_bullet_nums(sent): 
    return re.sub('^\s*\d+\n*[.\)]+\s*([^\d])', '\g<1>', sent)

def remove_leading_symbols(sent):
    return re.sub('^[^A-Za-z\"\'\d]+', '', sent)

def uppercase_first(sent): 
    return sent[0].upper() + sent[1:] if len(sent) != 0 else sent

In [ ]:
def tokenize_sent(text):
    doc = nlp(text, disable=['ner', 'attribute_ruler', 'lemmatizer', 'sentencizer'])
    sents = [str(sent).strip() for sent in doc.sents]
    return sents

In [ ]:
def lowercase(text):
    return text.lower()

def expand_contractions(text):
    for key in contractions:
        value = contractions[key]
        text = text.replace(key, value)
    return text

# remove digits 
def remove_digits(text): 
    return re.sub('\d+', ' ', text)

# remove symbols 
def remove_symbols(text):
    return re.sub('[^A-Za-z,.\s\d]+', ' ', text)

# lemmatization with spacy 
def lemmatize_text(text): 
    doc = nlp(text, disable=['parser','ner'])
    lemma = [token.lemma_ for token in doc if token.pos_ != 'PUNCT']
    return ' '.join(lemma)

# remove stop words 
def remove_stopwords(text, word_list=[]):
    stop_words = stopwords.words("english")
    stop_words.extend(word_list)
    stop_words = set(stop_words)
    return ' '.join(e.lower() for e in text.split() if e.lower() not in stop_words)

def get_extra_stopwords(game_name): 
    stopwords = set(['game'])
    doc = nlp(game_name.lower(), disable=['parser', 'ner'])
    for token in doc: 
        if token.pos_ not in {'PUNCT', 'NUM'}:
            stopwords.add(token.text)
    return stopwords

In [ ]:
sent_tuples = []

for game_id in df_reviews['game_id'].unique():
    with conn: 
        game_name = cursor.execute("""SELECT game_name FROM games WHERE game_id=?;""", (int(game_id),)).fetchone()[0]
    
    extra_stopwords = get_extra_stopwords(game_name)
    
    df_reviews_game = df_reviews[df_reviews['game_id'] == game_id]
    
    reviews_game_cleaned = df_reviews_game['review'].map(add_missing_punct)\
                    .map(replace_bullets)\
                    .map(remove_url)\
                    .map(remove_html_tags)\
                    .map(normalize_single_quote)\
                    .map(remove_non_ascii)\
                    .map(remove_ansi_escape_sequences)\
                    .map(remove_multi_whitespaces)
    
    for review_id, review in zip(df_reviews_game['review_id'], reviews_game_cleaned):
        sents = pd.Series(tokenize_sent(review)).map(remove_bullet_nums)\
                                                .map(remove_leading_symbols)\
                                                .map(uppercase_first)\
                                                .map(add_missing_punct)\
                                                .map(remove_multi_whitespaces)

        sents_prep = sents.map(lowercase)\
                        .map(expand_contractions)\
                        .map(remove_digits)\
                        .map(remove_symbols)\
                        .map(remove_multi_whitespaces)\
                        .map(lemmatize_text)\
                        .map(lambda x: remove_stopwords(x, word_list=extra_stopwords))

        for sent, sent_prep in zip(sents, sents_prep):
            sent_tuples.append((review_id, sent, sent_prep))

In [ ]:
with conn:
    cursor.executemany("""INSERT INTO sents (review_id, sent, sent_prep) VALUES 
                        (?, ?, ?);""", sent_tuples)    

### Step 3: 
- Use **review_prep** in table **reviews** to calculate special bigrams frequency, get 50 most frequent keywords.
- Insert **kw**, **freq** into table **kws**.
- Embed 50 keywords using S-BERT and cluster them using agglomerative clustering with a distance_threshold=0.6. 
	- Insert **cluster_name** (name of the most frequent keyword in cluster) into table **clusters**.
	- Insert **cluster_id** in table **kws**.

In [332]:
def get_n_gram(x, ngram, min_df=1):
    vec = CountVectorizer(ngram_range=[ngram, ngram], min_df=min_df).fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq

def bigramRules(bigram): 
    first_pos = set(['ADJ', 'NOUN'])
    second_pos = set(['NOUN'])
    
    tags = [token.pos_ for token in nlp(bigram, disable=['parser','ner'])]
    
    return tags[0] in first_pos and tags[1] in second_pos

In [329]:
df_sent_prep = pd.read_sql_query("""SELECT sent_prep, game_id
                    FROM sents JOIN reviews USING(review_id) JOIN games USING(game_id);""", conn)

In [336]:
get_n_gram(df_sent_prep['sent_prep'], 1, 3)

[('boss', 978),
 ('good', 869),
 ('soul', 849),
 ('metroidvania', 801),
 ('dark', 783),
 ('play', 772),
 ('art', 750),
 ('great', 683),
 ('like', 679),
 ('time', 520),
 ('fun', 520),
 ('fight', 513),
 ('style', 510),
 ('get', 487),
 ('one', 470),
 ('story', 387),
 ('design', 371),
 ('well', 368),
 ('difficulty', 367),
 ('music', 367),
 ('really', 358),
 ('hard', 351),
 ('feel', 347),
 ('make', 320),
 ('combat', 317),
 ('pixel', 300),
 ('gameplay', 291),
 ('beautiful', 284),
 ('amazing', 267),
 ('first', 262),
 ('lot', 257),
 ('challenge', 253),
 ('attack', 253),
 ('worth', 247),
 ('even', 236),
 ('enemy', 232),
 ('platformer', 230),
 ('would', 230),
 ('world', 226),
 ('little', 214),
 ('level', 213),
 ('character', 206),
 ('item', 206),
 ('go', 201),
 ('much', 200),
 ('take', 199),
 ('control', 197),
 ('soundtrack', 196),
 ('hour', 193),
 ('bit', 186),
 ('easy', 179),
 ('also', 178),
 ('find', 177),
 ('silksong', 176),
 ('love', 172),
 ('price', 170),
 ('mode', 169),
 ('short', 165),
 

In [ ]:
cluster_tuples = []
kw_tuples = []

for game_id in df_sent_prep['game_id'].unique():
    sents_prep = df_sent_prep[df_sent_prep['game_id'] == game_id]['sent_prep']
    bigram_freq = get_n_gram(sents_prep, 2, 3)
    bigram_df = pd.DataFrame(bigram_freq, columns=['bigram', 'freq'])
    
    bigram_df_50 = []
    count = 0
    
    for bigram, freq in zip(bigram_df['bigram'], bigram_df['freq']): 
        if bigramRules(bigram):
            bigram_df_50.append((bigram, freq))

            count += 1

            if count == 50: 
                break
                
    bigram_df = pd.DataFrame(bigram_df_50, columns=['bigram', 'freq'])
    
    kws = bigram_df['bigram']
    kw_embeddings = embedder.encode(kws)
    
    # Normalize the embeddings to unit length
    kw_embeddings = kw_embeddings /  np.linalg.norm(kw_embeddings, axis=1, keepdims=True)
    
    # perform agglomerative clustering
    clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=0.6)
    clustering_model.fit(kw_embeddings)
    cluster_assignment = clustering_model.labels_
    
    kw_clusters = {}
    for kw_id, cluster_id in enumerate(cluster_assignment):
        if cluster_id not in kw_clusters:
            kw_clusters[cluster_id] = []

        kw_clusters[cluster_id].append(kws[kw_id])
        
    bigram_df['cluster_num'] = cluster_assignment
    
    for cluster_num, cluster_val in kw_clusters.items():
        cluster_tuples.append((int(game_id), int(cluster_num), cluster_val[0]))
    
    for kw, freq, cluster_num in zip(bigram_df['bigram'], bigram_df['freq'], bigram_df['cluster_num']):
        kw_tuples.append((kw, freq, int(cluster_num), int(game_id)))

In [ ]:
with conn:
    cursor.executemany("""INSERT INTO clusters (game_id, cluster_num, cluster_name) VALUES 
                        (?, ?, ?);""", cluster_tuples)    

In [ ]:
with conn:
    cursor.executemany("""INSERT INTO kws (kw, freq, cluster_id) VALUES 
                        (?, ?, (SELECT cluster_id FROM clusters WHERE cluster_num=? AND game_id=?));""",
                       kw_tuples)

### Step 4: 
- Loop through **sent_prep** in table **sents**, fuzzy-match each **kw** in table **kws**. 
    - Insert **cluster_id**, **sent_id** in table **clusters_sents** to link table **clusters** and **sents**.

In [ ]:
df_sent_prep = pd.read_sql_query("""SELECT game_id, sent_prep, sent_id
                    FROM sents JOIN reviews USING(review_id) JOIN games USING(game_id);""", conn)

df_kw = pd.read_sql_query("""SELECT game_id, cluster_id, kw 
                    FROM kws LEFT JOIN clusters USING(cluster_id);""", conn)

In [ ]:
cluster_sent_tuples = []

for game_id in df_sent_prep['game_id'].unique():
    sents_prep = df_sent_prep[df_sent_prep['game_id'] == game_id]
    kws = df_kw[df_kw['game_id'] == game_id]
    
    kw_clusters = {}
    for kw, cluster_id in zip(kws['kw'], kws['cluster_id']):
        if cluster_id not in kw_clusters:
            kw_clusters[cluster_id] = []

        kw_clusters[cluster_id].append(kw)
        
    for sent_id, sent_prep in zip(sents_prep['sent_id'], sents_prep['sent_prep']):
        for cluster_id, kws in kw_clusters.items():
            for kw in kws:
                matches = find_near_matches(kw, sent_prep, max_l_dist=1)

                if len(matches) != 0: 
                    cluster_sent_tuples.append((cluster_id, sent_id))
                    break

In [ ]:
with conn:
    cursor.executemany("""INSERT INTO clusters_sents  (cluster_id, sent_id) VALUES 
                        (?, ?);""", cluster_sent_tuples)

### Step 5:
- Remove all **sent_id** in table **sents** if they don't exist in table **clusters_sents**.
    - Insert **score_flair**, **score_vader**, **recommended**, **score_total**, **sent_embedding** in table **sents**.

In [ ]:
# remove all sentences that contains no keyword from table sents
with conn: 
    cursor.execute("""
        DELETE FROM sents 
        WHERE sent_id NOT IN (
            SELECT DISTINCT sent_id
            FROM clusters_sents);
    """)

In [8]:
df_sent = pd.read_sql_query("""SELECT sent_id, sent, recommended
                                FROM sents JOIN reviews USING(review_id);""", conn)

In [ ]:
df_sent

In [161]:
def get_score_flair(sent, threshold=0.9): 
    sent_flair = flair.data.Sentence(sent)
    sentiment_model_fast.predict(sent_flair)
    
    value_flair = sent_flair.labels[0].value
    score_flair = sent_flair.labels[0].score
    
    value_flair = 1 if value_flair == 'POSITIVE' else -1
    
    score_flair = value_flair*score_flair
    
    return 1 if score_flair > threshold else -1 if score_flair < -threshold else 0

In [350]:
sentis = df_sent['sent'].map(get_score_flair) 

In [9]:
sent_embeddings = embedder.encode(df_sent['sent'])

sent_embeddings = sent_embeddings /  np.linalg.norm(sent_embeddings, axis=1, keepdims=True)

sent_embeddings_str = [pickle.dumps(sent_embedding) for sent_embedding in sent_embeddings]

In [351]:
sent_tuples = []

for senti, sent_embedding_str, sent_id in zip(sentis, sent_embeddings_str, df_sent['sent_id']): 
    sent_tuples.append((senti, sent_embedding_str, sent_id))     

In [354]:
with conn:
    cursor.executemany("""UPDATE sents SET (senti, sent_embedding) = (?, ?)
                        WHERE sent_id=?;""", sent_tuples)

In [12]:
sent_tuples = []

for sent_embedding_str, sent_id in zip(sent_embeddings_str, df_sent['sent_id']): 
    sent_tuples.append((sent_embedding_str, sent_id))

In [13]:
with conn:
    cursor.executemany("""UPDATE sents SET sent_embedding = ?
                        WHERE sent_id=?;""", sent_tuples)

In [22]:
df_sent_embedding = pd.read_sql_query("""SELECT sent_embedding FROM sents;""", conn)

In [23]:
df_sent_embeddings = np.array([pickle.loads(sent_embedding) for sent_embedding in df_sent_embedding['sent_embedding']])

In [25]:
df_sent_embeddings

array([[-0.04324087,  0.02180931,  0.01068075, ...,  0.03434775,
        -0.04854265, -0.00114909],
       [-0.03150932, -0.00418032,  0.04931263, ...,  0.06524638,
        -0.02565461,  0.02001331],
       [-0.05950123,  0.02035298,  0.03827898, ..., -0.00917198,
        -0.0430759 ,  0.03698024],
       ...,
       [ 0.01159374,  0.00428933, -0.00926146, ...,  0.00592567,
        -0.05686782,  0.05610381],
       [ 0.05544416,  0.03463249, -0.03918877, ..., -0.0102831 ,
        -0.02068716, -0.05278462],
       [ 0.03634208, -0.00284135,  0.07634094, ...,  0.03640819,
         0.01460779, -0.12906875]], dtype=float32)

In [357]:
df_sent = pd.read_sql_query("""SELECT sent_id, sent, senti
                                FROM sents;""", conn)

In [358]:
df_sent

,sent_id,sent,senti
0,1,Metroidvania with some influences from Dark So...,0
1,3,I had to check some guide to see the true ending.,0
2,21,A short but sweet metroidvania that is worth y...,1
3,25,It's a 2-d action platformer with a simple yet...,1
4,28,"Other than that, there isn't a ton of replay v...",-1
...,...,...,...
5539,34989,The few bosses I have fought have been extreme...,0
5540,35001,There's also free DLC.,0
5541,35003,If you like metroid-vanias that are a nice cha...,1
5542,35005,"Its atmosphere, its map, its bossfights, and e...",1
